![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2Farchitectures%2Ftracking%2Fsetup%2Fgithub&file=GitHub+Metrics+-+1+-+Commits+-+Initial+Creation.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/github/GitHub%20Metrics%20-%201%20-%20Commits%20-%20Initial%20Creation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2Farchitectures%2Ftracking%2Fsetup%2Fgithub%2FGitHub%2520Metrics%2520-%25201%2520-%2520Commits%2520-%2520Initial%2520Creation.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/github/GitHub%20Metrics%20-%201%20-%20Commits%20-%20Initial%20Creation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/architectures/tracking/setup/github/GitHub%20Metrics%20-%201%20-%20Commits%20-%20Initial%20Creation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# GitHub Metrics: Commit History Collection
For repository [/statmike/vertex-ai-mlops](https://www.github.com/statmike/vertex-ai-mlops)

Using the [GitHub API](https://docs.github.com/en/rest/commits/commits?apiVersion=2022-11-28) to:
- get commit history

**Notes:**

The API offers different ways to retrieve commit information.  These are different summaries of the the same information.
- `/commits` retrieves all commits for a repository, but not files
- `/commits/sha` where `sha` is the sha for a commit will retrieve the files associated with the individual commit
- `/contributors` a weekly summary of commits, additions, deletions for each contributor
- `/stats/code_frequency` is a weekly history of additions and deletions summarized
- `/stats/commit_activity` has the last year of weekly data for commits 

The most granualar levels is using `/commits` and then `/commits/sha` to retrieve timestamped file level information within commits.  This data could then be rolled up to all other levels in reporting.

Approach notes:
- I prefer to not convert date/times to formats in pandas and instead save this as a step in BigQuery.  Why? Loading a dataframe to BigQuery has a middle layer where the data gets serialized and transferred.  This middle step is another set of format conversions that can impact dates/times.  This can cause errors when later appending to the same BigQuery tables even when the dataframe matches the original identically. A -> B -> C is not the same as A -> B|C -> C

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/architectures/tracking/setup/github/GitHub%20Metrics%20-%201%20-%20Commits%20-%20Initial%20Creation.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [ ]:
PROJECT_ID = 'vertex-ai-mlops-369716' # replace with project ID

In [ ]:
try:
    import google.colab
    try:
      from google.cloud import secretmanager
    except ImportError:
      !pip install google-cloud-secret-manager -q
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

Updated property [core/project].


---
## Setup

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'vertex-ai-mlops-369716'

In [ ]:
REGION = 'us-central1'

github_user = 'statmike'
github_repo = 'vertex-ai-mlops'

BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'github_metrics'

In [ ]:
import requests
import json
import time
from datetime import datetime
import pandas as pd
from io import StringIO
import os, shutil

from google.cloud import bigquery
from google.cloud import secretmanager

In [ ]:
bq = bigquery.Client(project = PROJECT_ID)
secret_client = secretmanager.SecretManagerServiceClient()

In [ ]:
secret = secret_client.access_secret_version(request = {"name": f'projects/{PROJECT_ID}/secrets/github_api/versions/latest'})
pat = secret.payload.data.decode('utf-8')

---
## GitHub API

Define the API url for the user and repository.  Create a helper function that will make get request from API addresses and if the receive a 202 response (accepted request) then retry until it receives a 200 response (successful response).

In [ ]:
github_api_url = f'https://api.github.com/repos/{github_user}/{github_repo}'

In [ ]:
def metric_get(metric_type):
  response = requests.get(f'{github_api_url}/{metric_type}', headers = {'Authorization': f'Bearer {pat}', 'Accept': 'application/vnd.github+json'})
  while response.status_code == 202:
      time.sleep(10)
      response = requests.get(f'{github_api_url}/{metric_type}', headers = {'Authorization': f'Bearer {pat}', 'Accept': 'application/vnd.github+json'})
  return response

---
## Data Exploration

The following subsection retrieve and format data from different parts of the API related to commits.

---
### stats/code_frequency
- stats/code_frequency
- https://docs.github.com/en/rest/metrics/statistics?apiVersion=2022-11-28#get-the-weekly-commit-activity
- Full history of repository
- schema: list of list for each week
  - 3 elements are integers: UNIX Timestamp for 12AM each Sunday, additions, deletions
- LRO may return status_code = 202 while running, do retry until 200


In [ ]:
metric_type = 'stats/code_frequency'
response = metric_get(metric_type)
response.status_code

200

In [ ]:
code_frequency = pd.DataFrame(json.loads(response.text), columns = ['week', 'additions', 'deletions'])
#code_frequency['week'] = pd.to_datetime(code_frequency['week'], unit = 's')

In [ ]:
code_frequency.head()

,week,additions,deletions
0,1616889600,2983,-547
1,1617494400,7461,-3499
2,1618099200,12394,-6314
3,1618704000,7904,-6179
4,1619308800,0,0


---
### stats/commit_activity
- stats/commit_activity
- https://docs.github.com/en/rest/metrics/statistics?apiVersion=2022-11-28#get-the-last-year-of-commit-activity
- last year of commit activity
- schema: list of dict for each week
  - week: UNIX Timestamp for 12AM each Sunday
  - total: int of number of commits
  - days: list of ints with commit per day [sunday, ..., saturday]
- LRO may return status_code = 202 while running, do retry until 200


In [ ]:
metric_type = 'stats/commit_activity'
response = metric_get(metric_type)
response.status_code

200

In [ ]:
commit_activity = pd.DataFrame(json.loads(response.text)).drop(columns = ['days']).rename(columns = {'total' : 'commits'})
#commit_activity['week'] = pd.to_datetime(commit_activity['week'], unit = 's')

In [ ]:
commit_activity.head()

,commits,week
0,0,1645920000
1,10,1646524800
2,14,1647129600
3,6,1647734400
4,7,1648339200


---
### stats/contributors
- stats/contributors
- https://docs.github.com/en/rest/metrics/statistics?apiVersion=2022-11-28#get-all-contributor-commit-activity
- full history of repository
- schema: list of dict for contributor
  - total: total commits all time
  - weeks: list of dicts for each week:
    - w: UNIX Timestamp for 12AM each Sunday
    - a: additions
    - d: deletions
    - c: commits
  - author: dict
    - login: user name on GitHub
    - ...
- LRO may return status_code = 202 while running, do retry until 200


In [ ]:
metric_type = 'stats/contributors'
response = metric_get(metric_type)
response.status_code

200

In [ ]:
parsed_response = []
for e in json.loads(response.text):
  for week in e['weeks']:
    parsed_response += [{'author': e['author']['login'], 'week': week['w'], 'additions': week['a'], 'deletions': -1*week['d'], 'commits': week['c']}]

In [ ]:
parsed_response[0]

{'author': 'karticn-google',
 'week': 1616889600,
 'additions': 0,
 'deletions': 0,
 'commits': 0}

In [ ]:
contributors = pd.DataFrame(parsed_response)
#contributors['week'] = pd.to_datetime(contributors['week'], unit = 's')

In [ ]:
contributors.head()

,author,week,additions,deletions,commits
0,karticn-google,1616889600,0,0,0
1,karticn-google,1617494400,0,0,0
2,karticn-google,1618099200,0,0,0
3,karticn-google,1618704000,0,0,0
4,karticn-google,1619308800,0,0,0


### commits
- /commits
- Get a list of commits with details
- schema: list of dict for each commit


In [ ]:
page_size = 100
page = 1
raw_commits = []
while page_size == 100:
  response = requests.get(f'{github_api_url}/commits?per_page=100&page={page}', headers = {'Authorization': f'Bearer {pat}', 'Accept': 'application/vnd.github+json'})
  new_page = json.loads(response.text)
  if response.status_code == 200:
    raw_commits += new_page
    page_size = len(new_page)
    page +=1
  else: break

In [ ]:
len(raw_commits)

708

In [ ]:
raw_commits[0]

{'sha': '877ff535c3f01b9383a0cbb4be204705230983cc',
 'node_id': 'C_kwDOFwjp5toAKDg3N2ZmNTM1YzNmMDFiOTM4M2EwY2JiNGJlMjA0NzA1MjMwOTgzY2M',
 'commit': {'author': {'name': 'Mike Henderson',
   'email': 'statmike@gmail.com',
   'date': '2023-02-19T18:19:54Z'},
  'committer': {'name': 'Mike Henderson',
   'email': 'statmike@gmail.com',
   'date': '2023-02-19T18:19:54Z'},
  'message': 'notes',
  'tree': {'sha': '8dad1a7b6c6c707565dd8fc45561d1e73efb0666',
   'url': 'https://api.github.com/repos/statmike/vertex-ai-mlops/git/trees/8dad1a7b6c6c707565dd8fc45561d1e73efb0666'},
  'url': 'https://api.github.com/repos/statmike/vertex-ai-mlops/git/commits/877ff535c3f01b9383a0cbb4be204705230983cc',
  'comment_count': 0,
  'verification': {'verified': False,
   'reason': 'unsigned',
   'signature': None,
   'payload': None}},
 'url': 'https://api.github.com/repos/statmike/vertex-ai-mlops/commits/877ff535c3f01b9383a0cbb4be204705230983cc',
 'html_url': 'https://github.com/statmike/vertex-ai-mlops/commit/87

In [ ]:
commits = []
for i, c in enumerate(raw_commits):
  committer = c['commit']['committer']['name']
  author = c['commit']['author']['name']
  committer2 = ''
  if 'committer' in c and c['committer']:
    if 'login' in c['committer']: committer2 = c['committer']['login']
  author2 = ''
  if 'author' in c and c['author']:
    if 'login' in c['author']: author2 = c['author']['login']

  # refined author with logic:
  if author2: refined_author = author2
  else: refined_author = author 

  commits += [{
      'sha': c['sha'],
      'datetime': c['commit']['committer']['date'],
      'url': c['html_url'],
      'message': c['commit']['message'],
      'author': refined_author,
      #'committer': committer,
      #'author': author,
      #'committer2': committer2,
      #'author2': author2
  }]

In [ ]:
commits = pd.DataFrame(commits)

In [ ]:
commits.head()

,sha,datetime,url,message,author
0,877ff535c3f01b9383a0cbb4be204705230983cc,2023-02-19T18:19:54Z,https://github.com/statmike/vertex-ai-mlops/co...,notes,statmike
1,5b85c2efa0a17cf4b0a49d045dc259135a158c59,2023-02-19T01:19:08Z,https://github.com/statmike/vertex-ai-mlops/co...,process update,statmike
2,dbfc6a9755a8eb782856af435a8383c683a598c1,2023-02-19T01:17:43Z,https://github.com/statmike/vertex-ai-mlops/co...,add colab process,statmike
3,151476a18b3a56228e3810bcfd5416e7cfe0de5f,2023-02-19T01:13:21Z,https://github.com/statmike/vertex-ai-mlops/co...,GA4 reporting process update,statmike
4,e8c28c8ed407688435c470b3ce39e3b1d697eef3,2023-02-18T22:59:45Z,https://github.com/statmike/vertex-ai-mlops/co...,tracking update,statmike


In [ ]:
#commits['datetime'] = pd.to_datetime(commits['datetime'], infer_datetime_format = True)

In [ ]:
commits.head()

,sha,datetime,url,message,author
0,877ff535c3f01b9383a0cbb4be204705230983cc,2023-02-19T18:19:54Z,https://github.com/statmike/vertex-ai-mlops/co...,notes,statmike
1,5b85c2efa0a17cf4b0a49d045dc259135a158c59,2023-02-19T01:19:08Z,https://github.com/statmike/vertex-ai-mlops/co...,process update,statmike
2,dbfc6a9755a8eb782856af435a8383c683a598c1,2023-02-19T01:17:43Z,https://github.com/statmike/vertex-ai-mlops/co...,add colab process,statmike
3,151476a18b3a56228e3810bcfd5416e7cfe0de5f,2023-02-19T01:13:21Z,https://github.com/statmike/vertex-ai-mlops/co...,GA4 reporting process update,statmike
4,e8c28c8ed407688435c470b3ce39e3b1d697eef3,2023-02-18T22:59:45Z,https://github.com/statmike/vertex-ai-mlops/co...,tracking update,statmike


In [ ]:
commits.dtypes

sha         object
datetime    object
url         object
message     object
author      object
dtype: object

In [ ]:
commits.groupby(['author']).count()

,sha,datetime,url,message
author,,,,
Mike Henderson,44,44,44,44
PavelPetukhov,2,2,2,2
goodrules,10,10,10,10
karticn-google,1,1,1,1
mike henderson,1,1,1,1
statmike,650,650,650,650


In [ ]:
commits.loc[commits['author'].str.lower() == 'mike henderson', 'author'] = 'statmike'

In [ ]:
commits.groupby(['author']).count()

,sha,datetime,url,message
author,,,,
PavelPetukhov,2,2,2,2
goodrules,10,10,10,10
karticn-google,1,1,1,1
statmike,695,695,695,695


In [ ]:
commits.head()

,sha,datetime,url,message,author
0,877ff535c3f01b9383a0cbb4be204705230983cc,2023-02-19T18:19:54Z,https://github.com/statmike/vertex-ai-mlops/co...,notes,statmike
1,5b85c2efa0a17cf4b0a49d045dc259135a158c59,2023-02-19T01:19:08Z,https://github.com/statmike/vertex-ai-mlops/co...,process update,statmike
2,dbfc6a9755a8eb782856af435a8383c683a598c1,2023-02-19T01:17:43Z,https://github.com/statmike/vertex-ai-mlops/co...,add colab process,statmike
3,151476a18b3a56228e3810bcfd5416e7cfe0de5f,2023-02-19T01:13:21Z,https://github.com/statmike/vertex-ai-mlops/co...,GA4 reporting process update,statmike
4,e8c28c8ed407688435c470b3ce39e3b1d697eef3,2023-02-18T22:59:45Z,https://github.com/statmike/vertex-ai-mlops/co...,tracking update,statmike


### commits/files
- /commits/REF (use SHA)

In [ ]:
sha = list(commits['sha'])

In [ ]:
raw_files = []
for s in sha:
  page = 1
  response = requests.get(f'{github_api_url}/commits/{s}?per_page=100&page={page}', headers = {'Authorization': f'Bearer {pat}', 'Accept': 'application/vnd.github+json'})
  files = json.loads(response.text)['files']
  if len(files) == 100:
    while len(files) % 100 == 0:
      page += 1
      response = requests.get(f'{github_api_url}/commits/{s}?per_page=100&page={page}', headers = {'Authorization': f'Bearer {pat}', 'Accept': 'application/vnd.github+json'})
      files += json.loads(response.text)['files']
  raw_files += [{'sha': s, 'files': files}]

In [ ]:
len(raw_files)

708

In [ ]:
len(raw_files[2]['files'])

1

In [ ]:
raw_files[0]['files']

[{'sha': '2076f1aa518041c2a238e81ffc23e36fba2433d2',
  'filename': 'architectures/tracking/setup/github/readme.md',
  'status': 'modified',
  'additions': 1,
  'deletions': 0,
  'changes': 1,
  'blob_url': 'https://github.com/statmike/vertex-ai-mlops/blob/877ff535c3f01b9383a0cbb4be204705230983cc/architectures%2Ftracking%2Fsetup%2Fgithub%2Freadme.md',
  'raw_url': 'https://github.com/statmike/vertex-ai-mlops/raw/877ff535c3f01b9383a0cbb4be204705230983cc/architectures%2Ftracking%2Fsetup%2Fgithub%2Freadme.md',
  'contents_url': 'https://api.github.com/repos/statmike/vertex-ai-mlops/contents/architectures%2Ftracking%2Fsetup%2Fgithub%2Freadme.md?ref=877ff535c3f01b9383a0cbb4be204705230983cc',
  'patch': '@@ -7,5 +7,6 @@ Setup the data reads from the GitHub API\n \n ## Notebooks For Gathering and Processing Information:\n - [GitHub Metrics - 1 - Initial Creation](./GitHub%20Metrics%20-%201%20-%20Initial%20Creation.ipynb)\n+    - Building the tables `commits` and `commits_files` in the BigQuery

In [ ]:
commits_files = []
for c in raw_files:
  for f in c['files']:
    commits_files += [{
        'sha': c['sha'],
        'file_sha': f['sha'],
        'file': f"{github_user}/{github_repo}/{f['filename']}",
        'additions': f['additions'],
        'deletions': f['deletions']
    }]

In [ ]:
commits_files = pd.DataFrame(commits_files)

In [ ]:
commits_files.head()

,sha,file_sha,file,additions,deletions
0,877ff535c3f01b9383a0cbb4be204705230983cc,2076f1aa518041c2a238e81ffc23e36fba2433d2,statmike/vertex-ai-mlops/architectures/trackin...,1,0
1,5b85c2efa0a17cf4b0a49d045dc259135a158c59,d56dfb3c5ff1cb2bfd92ab404cd11a2677875618,statmike/vertex-ai-mlops/architectures/trackin...,1,0
2,dbfc6a9755a8eb782856af435a8383c683a598c1,696a6b8c109004639678b964a782fffea8bec850,statmike/vertex-ai-mlops/architectures/trackin...,11,0
3,151476a18b3a56228e3810bcfd5416e7cfe0de5f,ec4f76a6f89ae3b2e49be2761b7f8d21822ccec2,statmike/vertex-ai-mlops/architectures/trackin...,1,1
4,151476a18b3a56228e3810bcfd5416e7cfe0de5f,b2e135734a66e00479f69e8e4865c2e802e276c4,statmike/vertex-ai-mlops/architectures/trackin...,1,1


---
## Decision

Only keep `commits` and `commits_files` as they are more detailed and have fully history of the repository as well as including commits that are part of pull request.  The logic for the builds also includes loading comitts that arrive late due to being pulled from branches and other users or local clone days after they are comitted.

---
## Pandas Tables

The following section combines and does final formatting to prepare a dataframe version of data to save.

### commits

In [ ]:
commits = commits
commits.head()

,sha,datetime,url,message,author
0,877ff535c3f01b9383a0cbb4be204705230983cc,2023-02-19T18:19:54Z,https://github.com/statmike/vertex-ai-mlops/co...,notes,statmike
1,5b85c2efa0a17cf4b0a49d045dc259135a158c59,2023-02-19T01:19:08Z,https://github.com/statmike/vertex-ai-mlops/co...,process update,statmike
2,dbfc6a9755a8eb782856af435a8383c683a598c1,2023-02-19T01:17:43Z,https://github.com/statmike/vertex-ai-mlops/co...,add colab process,statmike
3,151476a18b3a56228e3810bcfd5416e7cfe0de5f,2023-02-19T01:13:21Z,https://github.com/statmike/vertex-ai-mlops/co...,GA4 reporting process update,statmike
4,e8c28c8ed407688435c470b3ce39e3b1d697eef3,2023-02-18T22:59:45Z,https://github.com/statmike/vertex-ai-mlops/co...,tracking update,statmike


### commits_files

In [ ]:
commits_files = pd.merge(commits, commits_files, on = 'sha', how = 'outer')
commits_files.head()

,sha,datetime,url,message,author,file_sha,file,additions,deletions
0,877ff535c3f01b9383a0cbb4be204705230983cc,2023-02-19T18:19:54Z,https://github.com/statmike/vertex-ai-mlops/co...,notes,statmike,2076f1aa518041c2a238e81ffc23e36fba2433d2,statmike/vertex-ai-mlops/architectures/trackin...,1,0
1,5b85c2efa0a17cf4b0a49d045dc259135a158c59,2023-02-19T01:19:08Z,https://github.com/statmike/vertex-ai-mlops/co...,process update,statmike,d56dfb3c5ff1cb2bfd92ab404cd11a2677875618,statmike/vertex-ai-mlops/architectures/trackin...,1,0
2,dbfc6a9755a8eb782856af435a8383c683a598c1,2023-02-19T01:17:43Z,https://github.com/statmike/vertex-ai-mlops/co...,add colab process,statmike,696a6b8c109004639678b964a782fffea8bec850,statmike/vertex-ai-mlops/architectures/trackin...,11,0
3,151476a18b3a56228e3810bcfd5416e7cfe0de5f,2023-02-19T01:13:21Z,https://github.com/statmike/vertex-ai-mlops/co...,GA4 reporting process update,statmike,ec4f76a6f89ae3b2e49be2761b7f8d21822ccec2,statmike/vertex-ai-mlops/architectures/trackin...,1,1
4,151476a18b3a56228e3810bcfd5416e7cfe0de5f,2023-02-19T01:13:21Z,https://github.com/statmike/vertex-ai-mlops/co...,GA4 reporting process update,statmike,b2e135734a66e00479f69e8e4865c2e802e276c4,statmike/vertex-ai-mlops/architectures/trackin...,1,1


---
## BigQuery Tables: Initial Creation


### commits

In [ ]:
commits_job = bq.load_table_from_dataframe(
    dataframe = commits,
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.commits"),
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE', # WRITE_TRUNCATE = replace if exists, WRITE_APPEND = append if exists, WRITE_EMPTY = write new but dont overwrite
        autodetect = True, # detect schema
    )
)
commits_job.result()

LoadJob<project=vertex-ai-mlops-369716, location=US, id=57fa4124-66df-449e-812f-bd0250b0d273>

### commits_files

In [ ]:
commits_files_job = bq.load_table_from_dataframe(
    dataframe = commits_files,
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.commits_files"),
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE', # WRITE_TRUNCATE = replace if exists, WRITE_APPEND = append if exists, WRITE_EMPTY = write new but dont overwrite
        autodetect = True, # detect schema
    )
)
commits_files_job.result()

LoadJob<project=vertex-ai-mlops-369716, location=US, id=9a2ec441-6d3e-44da-bd48-1eabdf263c5f>

---
## BigQuery Tables: Increment

- get last week of data: prior_weekly_commits
- keep all new records for 'last week' and newer: new_weekly_commits
- conditions:
  - if same number of rows and rows identitial: do nothing
  - elif rows are same for last week: remove it from new, if new records then append them
  - if row for last week differs: drop it from BQ, then append

### commits

Commits can arrive after they are create because a push or pull may be delayed.  For this reason, load the full commit history and check for new `sha` values.  Only these need to be added. This is always an append operation.

In [ ]:
page_size = 100
page = 1
raw_commits = []
while page_size == 100:
  response = requests.get(f'{github_api_url}/commits?per_page=100&page={page}', headers = {'Authorization': f'Bearer {pat}', 'Accept': 'application/vnd.github+json'})
  new_page = json.loads(response.text)
  if response.status_code == 200:
    raw_commits += new_page
    page_size = len(new_page)
    page +=1
  else: break

commits = []
for i, c in enumerate(raw_commits):
  committer = c['commit']['committer']['name']
  author = c['commit']['author']['name']
  committer2 = ''
  if 'committer' in c and c['committer']:
    if 'login' in c['committer']: committer2 = c['committer']['login']
  author2 = ''
  if 'author' in c and c['author']:
    if 'login' in c['author']: author2 = c['author']['login']

  # refined author with logic:
  if author2: refined_author = author2
  else: refined_author = author 

  commits += [{
      'sha': c['sha'],
      'datetime': c['commit']['committer']['date'],
      'url': c['html_url'],
      'message': c['commit']['message'],
      'author': refined_author,
      #'committer': committer,
      #'author': author,
      #'committer2': committer2,
      #'author2': author2
  }]

commits = pd.DataFrame(commits)
#commits['datetime'] = pd.to_datetime(commits['datetime'], infer_datetime_format = True)
commits.loc[commits['author'].str.lower() == 'mike henderson', 'author'] = 'statmike'

In [ ]:
commits.head()

,sha,datetime,url,message,author
0,09404bb01b5c98b55b2387ef5cf48fb73581bc18,2023-02-19T19:14:01Z,https://github.com/statmike/vertex-ai-mlops/co...,notes,statmike
1,877ff535c3f01b9383a0cbb4be204705230983cc,2023-02-19T18:19:54Z,https://github.com/statmike/vertex-ai-mlops/co...,notes,statmike
2,5b85c2efa0a17cf4b0a49d045dc259135a158c59,2023-02-19T01:19:08Z,https://github.com/statmike/vertex-ai-mlops/co...,process update,statmike
3,dbfc6a9755a8eb782856af435a8383c683a598c1,2023-02-19T01:17:43Z,https://github.com/statmike/vertex-ai-mlops/co...,add colab process,statmike
4,151476a18b3a56228e3810bcfd5416e7cfe0de5f,2023-02-19T01:13:21Z,https://github.com/statmike/vertex-ai-mlops/co...,GA4 reporting process update,statmike


In [ ]:
prior_commits = bq.query(query = f"""SELECT sha FROM `{BQ_PROJECT}.{BQ_DATASET}.commits`""").to_dataframe()

In [ ]:
new_commits = pd.merge(prior_commits, commits, on = 'sha', how = 'outer', indicator = True)
new_commits = new_commits[new_commits['_merge'] == 'right_only'].drop('_merge', axis = 1)

In [ ]:
new_commits.shape[0]

0

In [ ]:
if new_commits.shape[0] > 0:
  new_commits_job = bq.load_table_from_dataframe(
      dataframe = new_commits,
      destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.commits"),
      job_config = bigquery.LoadJobConfig(
          write_disposition = 'WRITE_APPEND', # WRITE_TRUNCATE = replace if exists, WRITE_APPEND = append if exists, WRITE_EMPTY = write new but dont overwrite
          autodetect = True # detect schema
      )
  )
  new_commits_job.result()

### commits_files

Only need to fetch files for new commits found above.  This is always an append operation.

In [ ]:
if new_commits.shape[0] > 0:
  sha = list(new_commits['sha'])
  raw_files = []
  for s in sha:
    page = 1
    response = requests.get(f'{github_api_url}/commits/{s}?per_page=100&page={page}', headers = {'Authorization': f'Bearer {pat}', 'Accept': 'application/vnd.github+json'})
    files = json.loads(response.text)['files']
    if len(files) == 100:
      while len(files) % 100 == 0:
        page += 1
        response = requests.get(f'{github_api_url}/commits/{s}?per_page=100&page={page}', headers = {'Authorization': f'Bearer {pat}', 'Accept': 'application/vnd.github+json'})
        files += json.loads(response.text)['files']
    raw_files += [{'sha': s, 'files': files}]

  commits_files = []
  for c in raw_files:
    for f in c['files']:
      commits_files += [{
          'sha': c['sha'],
          'file_sha': f['sha'],
          'file': f"{github_user}/{github_repo}/{f['filename']}",
          'additions': f['additions'],
          'deletions': f['deletions']
      }]
  commits_files = pd.DataFrame(commits_files)
  commits_files = pd.merge(new_commits, commits_files, on = 'sha', how = 'inner')

  new_commits_files_job = bq.load_table_from_dataframe(
      dataframe = commits_files,
      destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.commits_files"),
      job_config = bigquery.LoadJobConfig(
          write_disposition = 'WRITE_APPEND', # WRITE_TRUNCATE = replace if exists, WRITE_APPEND = append if exists, WRITE_EMPTY = write new but dont overwrite
          autodetect = True, # detect schema
      )
  )
  new_commits_files_job.result()

---
## Diagnostics

Mainly, can:
- the results in commits be created from commits_files

In [ ]:
# total commits in commits
query = f"""
SELECT count(*)
FROM `{BQ_PROJECT}.{BQ_DATASET}.commits`
"""
bq.query(query).to_dataframe()

,f0_
0,709


In [ ]:
# total commits in commits_files
query = f"""
SELECT count(*)
FROM
(SELECT DISTINCT sha FROM `{BQ_PROJECT}.{BQ_DATASET}.commits_files`)
"""
bq.query(query).to_dataframe()

,f0_
0,709
